# GEM and TSFC

In [ ]:
from os import path
import numpy
import loopy as lp

from firedrake.petsc import PETSc
from firedrake.utils import IntType, as_cstr, ScalarType, ScalarType_c, complex_mode, RealType_c

import ufl
import finat.ufl
from ufl.corealg.map_dag import map_expr_dag

import gem
import gem.impero_utils as impero_utils

import tsfc
import tsfc.kernel_interface.firedrake_loopy as firedrake_interface
import tsfc.ufl_utils as ufl_utils


@PETSc.Log.EventDecorator()
def to_reference_coords_newton_step(ufl_coordinate_element, parameters, x0_dtype="double", dX_dtype=ScalarType):
    # Set up UFL form
    cell = ufl_coordinate_element.cell
    domain = ufl.Mesh(ufl_coordinate_element)
    K = ufl.JacobianInverse(domain)
    x = ufl.SpatialCoordinate(domain)
    x0_element = finat.ufl.VectorElement("Real", cell, 0)
    x0 = ufl.Coefficient(ufl.FunctionSpace(domain, x0_element))
    from firedrake import conditional, as_vector
    y0 = conditional(x[0] > 1, as_vector([-100, -100]), x0)
    expr = ufl.dot(K, x - y0)

    # Translation to GEM
    C = ufl.Coefficient(ufl.FunctionSpace(domain, ufl_coordinate_element))
    expr = ufl_utils.preprocess_expression(expr, complex_mode=complex_mode)
    expr = ufl_utils.simplify_abs(expr, complex_mode)

    builder = firedrake_interface.KernelBuilderBase(ScalarType)
    builder.domain_coordinate[domain] = C

    Cexpr = builder._coefficient(C, "C")
    x0_expr = builder._coefficient(x0, "x0")
    loopy_args = [
        lp.GlobalArg(
            "C", dtype=ScalarType, shape=(numpy.prod(Cexpr.shape, dtype=int),)
        ),
        lp.GlobalArg(
            "x0", dtype=x0_dtype, shape=(numpy.prod(x0_expr.shape, dtype=int),)
        ),
    ]

    dim = cell.topological_dimension()
    point = gem.Variable('X', (dim,))
    loopy_args.append(lp.GlobalArg("X", dtype=ScalarType, shape=(dim,)))
    context = tsfc.fem.GemPointContext(
        interface=builder,
        ufl_cell=cell,
        point_indices=(),
        point_expr=point,
        scalar_type=parameters["scalar_type"]
    )
    translator = tsfc.fem.Translator(context)
    ir = map_expr_dag(translator, expr)

    # Unroll result
    ir = [gem.Indexed(ir, alpha) for alpha in numpy.ndindex(ir.shape)]

    # Unroll IndexSums
    max_extent = parameters["unroll_indexsum"]
    if max_extent:
        def predicate(index):
            return index.extent <= max_extent
        ir = gem.optimise.unroll_indexsum(ir, predicate=predicate)

    # Translate to loopy
    ir = impero_utils.preprocess_gem(ir)
    return_variable = gem.Variable('dX', (dim,))
    loopy_args.append(lp.GlobalArg("dX", dtype=dX_dtype, shape=(dim,)))
    assignments = [(gem.Indexed(return_variable, (i,)), e)
                   for i, e in enumerate(ir)]
    impero_c = impero_utils.compile_gem(assignments, ())
    kernel, _ = tsfc.loopy.generate(
        impero_c, loopy_args, ScalarType,
        kernel_name="to_reference_coords_newton_step")
    return lp.generate_code_v2(kernel).device_code()


In [ ]:
from firedrake import RectangleMesh
mesh = RectangleMesh(10, 10, 1, 1)

ufl_coordinate_element = mesh.ufl_coordinate_element()
parameters = tsfc.default_parameters()
print(to_reference_coords_newton_step(ufl_coordinate_element, parameters))